In [1]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu") 


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Dataset versions/JEE-ans-no_num-no_fillBlank-replaced_ofthefollowing-removed_one_letter_sent_also.csv")
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,question_id,question,syllabus_id,answer,question_type,len,words
0,0,2073346,how many π bonds are present in ferrocene,700295,a,Subjective Question,41,8
1,1,2073324,what is covalency of chlorine atom in second e...,698013,a,Subjective Question,58,10
2,2,2073225,in solid x atoms goes to corner of the cube an...,700078,No of x atoms 8 18 2 12 2,Subjective Question,126,25
3,3,2073215,calculate value of x y here x and y are the to...,697895,stryde images 2022 04 11 202204110162130016496...,Subjective Question,156,28
4,4,2073202,in phosphorus acid if x is number of non bondi...,697895,stryde images 2022 04 11 202204119693370016496...,Subjective Question,143,32
...,...,...,...,...,...,...,...,...
114798,117064,420545,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21
114799,117065,420544,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21
114800,117066,420543,subjective english question,-1,answer_subjective,Subjective Question,27,3
114801,117067,415580,question subjective english,-1,answer subjective hindi,Subjective Question,27,3


In [4]:
!pip install transformers==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install tensorflow==1.13.1
! pip install tensorflow-hub==0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [7]:
data["question"] = data["question"].apply(lambda x: clean_sentence(x))
data

,Unnamed: 0,question_id,question,syllabus_id,answer,question_type,len,words
0,0,2073346,how many π bonds are present in ferrocene,700295,a,Subjective Question,41,8
1,1,2073324,what is covalency of chlorine atom in second e...,698013,a,Subjective Question,58,10
2,2,2073225,in solid x atoms goes to corner of the cube an...,700078,No of x atoms 8 18 2 12 2,Subjective Question,126,25
3,3,2073215,calculate value of x y here x and y are the to...,697895,stryde images 2022 04 11 202204110162130016496...,Subjective Question,156,28
4,4,2073202,in phosphorus acid if x is number of non bondi...,697895,stryde images 2022 04 11 202204119693370016496...,Subjective Question,143,32
...,...,...,...,...,...,...,...,...
114798,117064,420545,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21
114799,117065,420544,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21
114800,117066,420543,subjective english question,-1,answer_subjective,Subjective Question,27,3
114801,117067,415580,question subjective english,-1,answer subjective hindi,Subjective Question,27,3


In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [9]:
import tensorflow_hub as hub
import tensorflow as tf
class UseSentenceEmbedding():
    def __init__(self):
        # g = tf.Graph()
        with tf.device('/CPU:0'):
        # We will be feeding 1D tensors of text into the graph.
            self.text_input = tf.placeholder(dtype=tf.string, shape=[None])
            
            #kindly replace the location in hub.module with the url commented out below

            # "https://tfhub.dev/google/universal-sentence-encoder-large/3"
            embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
            self.embedded_text = embed(self.text_input)
            init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
        # g.finalize()

        self.session = tf.Session(config=tf.ConfigProto( allow_soft_placement=True))
        self.session.run(init_op)
        print("init _____")



    def get_tokenized_sents_embeddings_USE(self, sents,expand=False):

           
        vectors_USE =  self.session.run(self.embedded_text, feed_dict={self.text_input: sents})

        return vectors_USE

use_embedding = UseSentenceEmbedding()

INFO:absl:hub.KerasLayer is not available because TensorFlow version is less than 1.14
INFO:absl:Using /tmp/tfhub_modules to cache modules.


init _____


In [10]:
import numpy as np
!pip install inflection


import inflection

from nltk.stem import PorterStemmer 
ps = PorterStemmer()
from gzip import open as gopen
from pandas.core.common import flatten
import gensim.models.poincare as poincare
def get_cleaned_taxonomy(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = ' '.join(value.split(">>"))
      # taxonomy_words = [inflection.singularize(val)  for token in value for val in token.split(" ") if val.isalpha()]
      cleaned_taxonomy.append( value )
  return cleaned_taxonomy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
qc_science_data = pd.read_csv("train_taxonomy_prediction.csv")
label_set = list(set(qc_science_data["board_syllabus"].values))

In [12]:
len(label_set)

312

In [13]:
targets = get_cleaned_taxonomy(label_set)
len(targets)

312

In [14]:
taxonomy_vectors = use_embedding.get_tokenized_sents_embeddings_USE(targets)
taxonomy_vectors.shape

(312, 512)

In [15]:
train_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)

In [16]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split


def get_tokenized_input(text):
    input_ids = []
    attention_masks = []

    for sent in text:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 128,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            truncation=True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    # Print sentence 0, now as a list of IDs.
    print('Original: ', text[0])
    print('Token IDs:', input_ids[0])
    return input_ids, attention_masks

In [22]:
input_ids, attention_masks = get_tokenized_input(data["question"].values)

Original:  how many π bonds are present in ferrocene
Token IDs: tensor([  101,  2129,  2116,  1170,  9547,  2024,  2556,  1999, 10768, 18933,
        17968,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,


In [18]:

import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint

from tqdm import tqdm
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import torch.nn.functional as F

import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
cos_label = nn.CosineSimilarity(dim=1, eps=1e-5)

dist = torch.nn.PairwiseDistance(p=2.0, eps=1e-06)
nn.PairwiseDistance(p=2)
class MHSA(nn.Module):
  def __init__(self,
         emb_dim,
         kqv_dim,
         num_heads=2):
    super(MHSA, self).__init__()
    self.emb_dim = emb_dim
    self.kqv_dim = kqv_dim
    self.num_heads = num_heads

    self.w_k = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_q = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_v = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_out = nn.Linear(kqv_dim * num_heads, emb_dim)

  def forward(self, query, key, value):
    # print("query",query.shape)
    b, t = query.shape
    e = self.kqv_dim
    h = self.num_heads
    keys = self.w_k(key).view(b, h, e)
    values = self.w_v(value).view(b, h, e)
    queries = self.w_q(query).view(b, h, e)

    # keys = keys.transpose(2, 1)
    # queries = queries.transpose(2, 1)
    # values = values.transpose(2, 1)

    dot = queries @ keys.transpose(2, 1)  #(b*h*e) @ (b*e*h)
    dot = dot / np.sqrt(e)  # (b*h*h)
    dot = F.softmax(dot, dim=2)

    out = dot @ values   # (b*h*h) @ (b*h*e) = (b*h*e)
    out = out.contiguous().view(b, h * e)
    out = self.w_out(out)
    return out
# Neural Classifierwork

# Discussion TODOS
# try hierarhical interaction (TODO)

# try bringing in modalities (image, or video)

# Go from classical algorithm -> deep learning

class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=False)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(384, 512)
        self.multi_head_attention = MHSA(512, 512,8)
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim = 512,  num_heads = 8, batch_first=True)


    def forward(self,tokens,masks, targets=None, skip_attention=False):
        # print("tokens", tokens.shape)
        outputs = self.bert(tokens, attention_mask=masks)[2]
        # outputs[2] = outputs[2].permute(0,2,1)
        output_1 = outputs[-1].permute(1,0,2)
        # print(outputs[1].shape,outputs[0].shape)
        output_1 = torch.mean(output_1, dim=0)
        # output_2 = outputs[-2].permute(1,0,2)
        # output_2 = torch.mean(output_2, dim=0)
        # print("output_2", output_2.shape, output_1.shape)
        pooled_output = outputs[-1] #output_1 # torch.cat((output_1, output_2), dim=1)
        # print("pooled_output", pooled_output.shape)
        x = self.fc1(pooled_output)
        # x = self.fc2(x)
        # print("x shape",x.shape)
        targets_curr_batch = []
        for input_x in x:
            # print(input_x.shape)
            distance = cos_label(torch.mean(input_x,dim=0).reshape(1,-1), train_poincare_tensor)
            distances,indices = torch.topk(distance,1,largest=True)

            target_distances = (F.normalize(train_poincare_tensor[indices],p=2,dim=1) - F.normalize(train_poincare_tensor,p=2,dim=1)).pow(2).sum(1) #cos_label(unique_poincare_tensor[indices].reshape(1,-1), unique_poincare_tensor)
            distances,indices = torch.topk(target_distances,5,largest=False)
            targets_curr_batch.append(train_poincare_tensor[indices].reshape(1,5,512))
            # targets_curr_batch.append(unique_poincare_tensor[indices])
            # print("here")
        # print(len(targets_curr_batch))
        targets_batch = torch.cat(targets_curr_batch, dim=0)
        # print("targets_batch",targets_batch.shape)
        attn_output, attn_output_weights = self.multihead_attn(targets_batch, x, x)
        # target_attn_output, attn_output_weights = self.multihead_attn(x, targets_batch, targets_batch)

        x = torch.sum(attn_output,dim=1)
        # print("X shape",x.shape)
        return x

class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin

    def forward(self, output, target):
        loss=0
        for i in range(len(output)):
            v_image = F.normalize(output[i],p=2,dim=0)
            t_label = F.normalize(target[i],p=2,dim=0)

            for i in range(5):
                j = randint(0, len(output)-1)
                while j == i:
                    j = randint(0, len(output)-1)
                t_j = F.normalize(target[j],p=2,dim=0)
                loss+= torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
        return loss / (len(output)*5)


In [19]:
from transformers import BertModel, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, AdamW, BertConfig

# Loads BertModel, the pretrained BERT model with a single 
model = MulticlassClassifier('bert-base-uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/research_lo_content_taxonomy_classification/model_euclidean_USE_cos_final_attention_V3/model_weights'))

# Tell pytorch to run this model on the GPU.
model.cuda()


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

MulticlassClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [20]:
def get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor):
    model.eval()
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

    input_ids = input_ids.to('cuda')
    attention_masks = attention_masks.to('cuda')
    train_poincare_tensor = train_poincare_tensor.to('cuda')
    # Tracking variables 
    predictions , true_labels = [], []
    for input_id,attention_mask in zip(input_ids, attention_masks):
        with torch.no_grad():
            outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
            
        distances = cos(outputs,train_poincare_tensor)
        distances,indices = torch.topk(distances,3,largest=True)
        label_set = np.array(label_set)
        predictions.append(label_set[indices.cpu().numpy()])
        print(len(predictions))
    return predictions
    

In [23]:
train_poincare_tensor = train_poincare_tensor.to(device)
predictions = get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor)

Streaming output truncated to the last 5000 lines.
109804
109805
109806
109807
109808
109809
109810
109811
109812
109813
109814
109815
109816
109817
109818
109819
109820
109821
109822
109823
109824
109825
109826
109827
109828
109829
109830
109831
109832
109833
109834
109835
109836
109837
109838
109839
109840
109841
109842
109843
109844
109845
109846
109847
109848
109849
109850
109851
109852
109853
109854
109855
109856
109857
109858
109859
109860
109861
109862
109863
109864
109865
109866
109867
109868
109869
109870
109871
109872
109873
109874
109875
109876
109877
109878
109879
109880
109881
109882
109883
109884
109885
109886
109887
109888
109889
109890
109891
109892
109893
109894
109895
109896
109897
109898
109899
109900
109901
109902
109903
109904
109905
109906
109907
109908
109909
109910
109911
109912
109913
109914
109915
109916
109917
109918
109919
109920
109921
109922
109923
109924
109925
109926
109927
109928
109929
109930
109931
109932
109933
109934
109935
109936
109937
109938
1099

In [24]:
predictions

[array(['chemistry>>chemistry : part i>>coordination compounds',
        'chemistry>>chemistry : part i>>chemical bonding and molecular structure',
        'chemistry>>chemistry : part i>>surface chemistry'], dtype='<U116'),
 array(['physics>>physics : part - ii>>atoms',
        'physics>>physics : part - ii',
        'physics>>physics : part - ii>>dual nature of radiation and matter'],
       dtype='<U116'),
 array(['chemistry>>chemistry : part i>>electrochemistry',
        'chemistry>>chemistry : part i>>the solid state',
        'physics>>physics: part - i>>electrostatic potential and capacitance'],
       dtype='<U116'),
 array(['chemistry>>chemistry : part i>>chemical bonding and molecular structure',
        'chemistry>>chemistry : part i>>coordination compounds',
        'chemistry>>chemistry : part ii>>organic chemistry - some basic principles and techniques'],
       dtype='<U116'),
 array(['chemistry>>chemistry : part i>>chemical bonding and molecular structure',
        'che

In [25]:
final_predictions = []
for pred in predictions:
    pred_list = pred.tolist()
    predictions_1 = (',').join(pred_list)
    final_predictions.append(predictions_1)
final_predictions = pd.Series(final_predictions, dtype = np.str)
data["tagrec++_predictions"] = final_predictions
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,Unnamed: 0,question_id,question,syllabus_id,answer,question_type,len,words,tagrec++_predictions
0,0,2073346,how many π bonds are present in ferrocene,700295,a,Subjective Question,41,8,chemistry>>chemistry : part i>>coordination co...
1,1,2073324,what is covalency of chlorine atom in second e...,698013,a,Subjective Question,58,10,"physics>>physics : part - ii>>atoms,physics>>p..."
2,2,2073225,in solid x atoms goes to corner of the cube an...,700078,No of x atoms 8 18 2 12 2,Subjective Question,126,25,chemistry>>chemistry : part i>>electrochemistr...
3,3,2073215,calculate value of x y here x and y are the to...,697895,stryde images 2022 04 11 202204110162130016496...,Subjective Question,156,28,chemistry>>chemistry : part i>>chemical bondin...
4,4,2073202,in phosphorus acid if x is number of non bondi...,697895,stryde images 2022 04 11 202204119693370016496...,Subjective Question,143,32,chemistry>>chemistry : part i>>chemical bondin...
...,...,...,...,...,...,...,...,...,...
114798,117064,420545,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21,"computer science[c++]>>linked lists, stacks an..."
114799,117065,420544,a dice is rolled thrice the probability of get...,-1,frac 5 24,Single Choice Question,114,21,"computer science[c++]>>linked lists, stacks an..."
114800,117066,420543,subjective english question,-1,answer_subjective,Subjective Question,27,3,"social science>>economics>>consumer rights,pol..."
114801,117067,415580,question subjective english,-1,answer subjective hindi,Subjective Question,27,3,"social science>>economics>>consumer rights,soc..."


In [30]:
data["tagrec++_predictions"].values[:20]

array(['chemistry>>chemistry : part i>>coordination compounds,chemistry>>chemistry : part i>>chemical bonding and molecular structure,chemistry>>chemistry : part i>>surface chemistry',
       'physics>>physics : part - ii>>atoms,physics>>physics : part - ii,physics>>physics : part - ii>>dual nature of radiation and matter',
       'chemistry>>chemistry : part i>>electrochemistry,chemistry>>chemistry : part i>>the solid state,physics>>physics: part - i>>electrostatic potential and capacitance',
       'chemistry>>chemistry : part i>>chemical bonding and molecular structure,chemistry>>chemistry : part i>>coordination compounds,chemistry>>chemistry : part ii>>organic chemistry - some basic principles and techniques',
       'chemistry>>chemistry : part i>>chemical bonding and molecular structure,chemistry>>chemistry : part i>>chemical kinetics,chemistry>>chemistry : part i>>some basic concepts of chemistry',
       'chemistry>>chemistry : part i>>chemical bonding and molecular structure,c

In [26]:
data.to_csv("JEE_EM_tagrec_tagged.csv", index=False)

In [29]:
!cp JEE_EM_tagrec_tagged.csv "/content/drive/MyDrive/Dataset versions/"